# Setting up data collection 
### [RN] 2/7/2021 12:42 PM

I set up a github for this project. I will need to let the detector run for a long period of time, and have data update periodically for me and PHYS 150 students to look at. I plan on using git in the terminal of a raspberry pi to handle updates. 

I already have a detector and raspberry pi from Prof. Stuart that I will use to do coscmic ray detection. I am familiar with how this works and have already probed it to check its fucntionality. I will include this in my report, but I already have it done for previous work so I this notebook does not so much cover that. 

The detector basically works by sensing photons generated by scintillating material, and then generating an output signal for the rasperry pi to read. Two scintillating bars are inputted to and logic such that I will only get a signal if the bars are triggered at the same time. This is done because cosmic rays are very penetrating, whearas most background radiation is not penentrating enough to trigger stacked scinnitallors at the same time. This detector is light sensitive, so I will be keeping the whole thing covered in a box. It will stay in the corner of my room for the duration of collection. 

I will also need to scrape environmental data such as wind speed, temperature, and humidity to look for patterns in cosmic ray incident frequency. 


Becuase we are using this detector in the class i am LA'ing for, I already have scripts to get digitalized data out of the detector. 

In [1]:
### Code from Prof stuart to readout cosmic ray detector 

#!/usr/bin/env python3

# Starting example of reading the Stretch signal on pin 37 and recording the time and rate.
import numpy  as  np
import math
import sys
import os
import os.path
from os import path
import RPi.GPIO as GPIO
import time
import datetime

GPIO.setmode(GPIO.BOARD)
GPIO.setup(37,GPIO.IN)

if len(sys.argv) < 3:
    print("You need to pass the run number and max time in seconds")
    exit()
    
RunNum = int(sys.argv[1])
maxtime = int(sys.argv[2])
# Make sure that the run directory is there
if not path.exists("/home/pi/150/data/Run"+sys.argv[1]):
    os.system("mkdir /home/pi/150/data/Run"+sys.argv[1])    

# We will save data in three ways in different files. 
#   First is a time stamp of every event.
#   Second is the count of events per minute of livetime.
#   Third is the final, overall summary.
# We will pause after each minute of livetime to update plots.
eventsfile = "/home/pi/150/data/Run"+sys.argv[1]+"/events.dat"
minutesfile = "/home/pi/150/data/Run"+sys.argv[1]+"/minutesrates.dat"
summaryfile = "/home/pi/150/data/Run"+sys.argv[1]+"/summary.dat"
# Open output files.
eventsOut = open(eventsfile, "w")
minutesOut = open(minutesfile, "w")

# Define the stopfile; when it exists we end the run
stopfile = "/home/pi/150/db/stop"

# Determine how long it takes to measure the time
time1 = datetime.datetime.now()
for i in range(1,1000000):
    time2 = datetime.datetime.now()
    timediff2 = time2 - time1
print("Time required to measure time = ",timediff2.total_seconds(),"microseconds")

# Determine how long it takes to check if the stopfile exists
time1 = datetime.datetime.now()
done = 0
for i in range(1,1000000):
    if path.exists(stopfile):
        done = 1
time2 = datetime.datetime.now()
timediff2 = time2 - time1
print("Time required to check stop file = ",timediff2.total_seconds(),"microseconds")

# Auxiliary data is currently just zeros as placeholders.
ADC1 = 0 # Pulse height for channel 1 (reserved for future)
ADC2 = 0 # Pulse height for channel 2 (reserved for future)
AUX1 = 0 # Reserved for future use
AUX2 = 0 # Reserved for future use 
AUX3 = 0 # Reserved for future use 

# Initialize the event counters and timers
nCount = 0 # Counts since start of run
runStartTime = datetime.datetime.now()
timeSinceRunStart = datetime.datetime.now() - runStartTime
minuteStartTime = runStartTime
timeSinceMinuteStart = minuteStartTime - runStartTime
done = 0 # Set to 1 if the data taking should end
nMinuteCount = 0  # Counts in the last minute
nMinutes = 0 # Number of minutes since start of run
prevEventTime = datetime.datetime.now()
liveTime = 0  # Total live time, not counting processing and STRETCH time.

# Loop while checking for a high signal on pin 37
try:
    while done < 1:
        if maxtime > 0 and timeSinceRunStart.total_seconds() > maxtime :
            done = 1
        if path.exists(stopfile):
            done = 1
        if GPIO.input(37)==1:  # Check for a STRETCH signal
            # Have an event, process it
            currentTime = datetime.datetime.now()
            timeSinceRunStart = currentTime - runStartTime
            timeSinceMinuteStart = currentTime - minuteStartTime
            timeSincePrevEvent = currentTime - prevEventTime
            liveTime = liveTime + timeSincePrevEvent.total_seconds()
            nCount = nCount + 1
            nMinuteCount = nMinuteCount + 1
            print(nCount,timeSinceRunStart.total_seconds(), timeSincePrevEvent.total_seconds(), liveTime, currentTime.year, currentTime.month, currentTime.day, currentTime.hour, currentTime.minute, currentTime.second, currentTime.microsecond, ADC1, ADC2, AUX1, AUX2, AUX3, file=eventsOut, end="\n")
            if timeSinceMinuteStart.total_seconds() > 60:
                # A minute has passed, calculate the rate per minute, write it out, and update run summary
                nMinutes = nMinutes + 1
                print(nMinutes,nMinuteCount, file=minutesOut, end="\n")
                minutesOut.flush()
                eventsOut.flush()
                if nMinutes > 4:
                    os.system("CosmicRunSummary") # Make updated summary plots for monitoring
                minuteStartTime = datetime.datetime.now() # Start a new one minute clock
                nMinuteCount = 0 # Reset the count for the next minute
            while GPIO.input(37)==1: # Wait for the Stretch signal to fall
                time.sleep(0.00001)        
            prevEventTime = datetime.datetime.now()

finally:
    # Calculate total rate
    endTime = datetime.datetime.now()
    timeSinceRunStart = endTime - runStartTime
    nSeconds = timeSinceRunStart.total_seconds()
    rate = int(1000*nCount/nSeconds) # count rate in mHz; choose mHz to simplify as an integer
    rateErr = int(1000*math.sqrt(nCount)/nSeconds)
    
    # Now write summary information to the screen and a file
    print("")
    print("Raw count rate = ",nCount,"/",nSeconds,"=",rate,"+-",rateErr,"mHz")
    fileSummary = open(summaryfile, "w") 
    print(nCount,int(1000*nSeconds),rate,rateErr,file=fileSummary)
    print("NCount=",nCount," RunTime=",int(1000*nSeconds)," seconds Rate=",rate," &plusmn; ",rateErr," mHz")

    # Close and clean
    eventsOut.close() 
    minutesOut.close() 
    print("Run ended")
    GPIO.cleanup()
    fileSummary.close()
    os.system("CosmicRunSummary") # Make final summary plots
    # Remove the stopfile if it exists
    if path.exists(stopfile):
        os.system("rm -f "+stopfile) 

ModuleNotFoundError: No module named 'RPi'

### [RN] 12:48 

First I am copying this code onto my rapberry pi to see if it works to read out signals. I will need to modify it because I don't need a lot of the features that are written in for my purposes. I just want basic tabulated output data. 

Next I checked if I needed to install anything. My pi already has all the necessary packages. Moving on,  ill modify the code so that I just get lines of data.

Because I am going to leave this running for a long time, i am thinking for error handling purposes it will be better to use shell scripts to run data acqusition over and over again rather than just leave it in python. It will also be easier to scraping and data updates with shell scripting. 

In this case, I really only need a few basic lines from the python script Stuart provided. 

In [2]:
### First test run: ReadOut.py
#!/usr/bin/env python3


import numpy  as  np
import math
import sys
import os
import os.path
from os import path
import RPi.GPIO as GPIO
import time
import datetime
import sys

GPIO.setmode(GPIO.BOARD)
GPIO.setup(37,GPIO.IN)
n= 0
while n < 10:
    if GPIO.input(37)==1:
        currentTime = datetime.datetime.now()
        print(currentTime)
        n = n +1
        print(n)



ModuleNotFoundError: No module named 'RPi'

In [ ]:
### ReadOut.py, desktop in sunlight  

2021-02-07 12:58:34.895885
1
2021-02-07 12:58:34.895977
2
2021-02-07 12:58:34.896470
3
2021-02-07 12:58:34.896663
4
2021-02-07 12:58:34.896800
5
2021-02-07 12:58:34.896935
6
2021-02-07 12:58:34.897071
7
2021-02-07 12:58:34.897205
8
2021-02-07 12:58:34.897338
9
2021-02-07 12:58:34.897470
10

### [RN] 12:59 

My code works, but the signals are way too fast. This is probably because the detectors are in the light. Ill try covering them and running again. 


In [ ]:
### ReadOut.py output with detectors covered 

2021-02-07 13:00:44.187321
1
2021-02-07 13:00:44.187413
2
2021-02-07 13:00:44.187455
3
2021-02-07 13:00:44.187494
4
2021-02-07 13:00:44.187530
5
2021-02-07 13:00:44.187565
6
2021-02-07 13:00:44.187600
7
2021-02-07 13:00:44.187634
8
2021-02-07 13:00:44.187671
9
2021-02-07 13:00:44.187707
10

Still too fast. I need to change the threshold for counting a signal by manually messing around with the potentiometer on the discriminators. I also need to make sure both are set to the same position. There is a green light that shows events, so I can use this to visually set the dicsrimator so that signals are counted at a reasonable rate. This rate should not vary with small changes in the discrimator threshold to show that the signals are actually signals. However when the threshold is changed a certian amount, there should be all the sudden no signals, or constant signals the other way. This means I am looking for a region where the rate is relatively constant to signifiy singals are not artifact, and all the incidents are being counted. 

This setting of the threshold will contribute to systematic error. 

First I turned both variable restors completely clockwise until i met low resistance/ The green light is flashing very fast on AND mode. 

In [ ]:
### ReadOut.py output with pot all the way clockwise 

2021-02-07 13:09:38.909787
1
2021-02-07 13:09:38.909877
2
2021-02-07 13:09:38.909922
3
2021-02-07 13:09:38.909962
4
2021-02-07 13:09:38.910002
5
2021-02-07 13:09:38.910038
6
2021-02-07 13:09:38.910073
7
2021-02-07 13:09:38.910121
8
2021-02-07 13:09:38.910161
9
2021-02-07 13:09:38.910201
10

One full counter-clockwise turn still leaves the green light going off very fast. 

So does two. Three is less frequent 



In [ ]:
### ReadOut.py output with pot all the way clockwise, three full counterclockwise

2021-02-07 13:12:10.242341
1
2021-02-07 13:12:10.242432
2
2021-02-07 13:12:10.242474
3
2021-02-07 13:12:10.242513
4
2021-02-07 13:12:10.242557
5
2021-02-07 13:12:10.242595
6
2021-02-07 13:12:10.242630
7
2021-02-07 13:12:10.242667
8
2021-02-07 13:12:10.242704
9
2021-02-07 13:12:10.242740
10

Still way too fast. Something is wrong. 

I realized I forgot to add a sleep time to wait for the signal to die down and count the next one. 



In [ ]:
### First test run: ReadOut.py
#!/usr/bin/env python3


import numpy  as  np
import math
import sys
import os
import os.path
from os import path
import RPi.GPIO as GPIO
import time
import datetime
import sys

GPIO.setmode(GPIO.BOARD)
GPIO.setup(37,GPIO.IN)
n= 0
while n < 10:
    if GPIO.input(37)==1:
        currentTime = datetime.datetime.now()
        print(currentTime)
        n = n +1
        print(n)
    while GPIO.input(37) == 1:
        time.sleep(0.00001)

Now, I am 6 counter clockwise turns from all they way clockwise and im getting stuck on one count. Ill go back down to four turns from clockwise. i can see that the green light is flashing pretty slow, but im getting no readou. I need to adjust my script. 

apparently pin 37 never reads low. It looks like the strethc signal is actually reading out to GPIO21. This pin is not always high, which I can tell by asking GPIO(21). However the script is still getting stuck. 

It is actually GPIO19, I had my orientation flipped. 



In [ ]:
### This code shows me pin 19 is on and off as it should be rather than always 1. 

while n < 1000:
    print(GPIO.input(19))
    time.sleep(0.00001)
    n = n + 1

In [ ]:
### New ReadOut.py

#!/usr/bin/env python3
  
import numpy  as  np
import math
import sys
import os
import os.path
from os import path
import RPi.GPIO as GPIO
import time
import datetime
import sys

GPIO.setmode(GPIO.BOARD)
GPIO.setup(19,GPIO.IN)
n= 0

while n < 10:
    if GPIO.input(19)==1:
        currentTime = datetime.datetime.now()
        print(currentTime)
        n = n +1
        while GPIO.input(19) == 1:
            time.sleep(0.00001)
    else:
        print("not high")

Again, turning pot on discrimators all the way clockwise. I am goign to use jumper cables to attatch now because it is too hard to see whats going on with the board just sitting on the pi. 

### [RN] 2:03 
rewired with jumper cables. reading stretch out of pin37, getting reasonable results now. 


In [ ]:
## this is ReadOut.py now. 

#!/usr/bin/env python3
  
import numpy  as  np
import math
import sys
import os
import os.path
from os import path
import RPi.GPIO as GPIO
import time
import datetime
import sys

GPIO.setmode(GPIO.BOARD)
GPIO.setup(37,GPIO.IN)
n= 0
while n < 10:
    if GPIO.input(37)==1:
        currentTime = datetime.datetime.now()
        print(currentTime)
        n = n +1
        while GPIO.input(37)==1:
            time.sleep(0.00001)

With both pot turned all the way clockwise, I get this output

2021-02-07 14:05:35.245556
2021-02-07 14:05:35.449901
2021-02-07 14:05:36.300732
2021-02-07 14:05:36.666075
2021-02-07 14:05:38.430109
2021-02-07 14:05:41.304933
2021-02-07 14:05:42.266924
2021-02-07 14:05:45.247242
2021-02-07 14:05:45.278609
2021-02-07 14:05:46.335714

Each turn one counter clockwise 

2021-02-07 14:06:46.290807
2021-02-07 14:06:47.462306
2021-02-07 14:06:48.052634
2021-02-07 14:06:49.193342
2021-02-07 14:06:50.596241
2021-02-07 14:06:51.386265
2021-02-07 14:06:51.473096
2021-02-07 14:06:51.893717
2021-02-07 14:06:53.880926
2021-02-07 14:06:54.675571

I should see a difference if the detectors are  in light or covered. I will turn one more, total two counter clockwise from fully clockwise. 

Covered: 

2021-02-07 14:08:11.388889
2021-02-07 14:08:12.575267
2021-02-07 14:08:15.551328
2021-02-07 14:08:17.992875
2021-02-07 14:08:18.118338
2021-02-07 14:08:18.632534
2021-02-07 14:08:19.570835
2021-02-07 14:08:20.707040
2021-02-07 14:08:26.814856
2021-02-07 14:08:28.108450

In sunlight: 

2021-02-07 14:08:54.693511
2021-02-07 14:08:55.332817
2021-02-07 14:08:56.011253
2021-02-07 14:08:57.468392
2021-02-07 14:09:00.345751
2021-02-07 14:09:01.164575
2021-02-07 14:09:03.979456
2021-02-07 14:09:04.805602
2021-02-07 14:09:05.593744
2021-02-07 14:09:05.715711

clearly less frequent.

I will need to use good averaging and error anlysis to account for systematic erros due to the potentiometer. In looking for patterns, comparative studies will be better. 

Turning one more counterclockwise seems to give similar results. 

2021-02-07 14:14:33.613581
2021-02-07 14:14:35.797212
2021-02-07 14:14:36.628978
2021-02-07 14:14:37.103202
2021-02-07 14:14:38.896983
2021-02-07 14:14:40.247188
2021-02-07 14:14:40.834352
2021-02-07 14:14:42.603305
2021-02-07 14:14:42.756934
2021-02-07 14:14:42.765241

The scintillators have been horizontally next to eachtoher this whole time.  If I stack them, I get 

2021-02-07 14:15:31.641952
2021-02-07 14:15:32.653655
2021-02-07 14:15:32.712299
2021-02-07 14:15:35.850260
2021-02-07 14:15:36.426308
2021-02-07 14:15:36.500303
2021-02-07 14:15:37.686667
2021-02-07 14:15:38.777775
2021-02-07 14:15:39.016402
2021-02-07 14:15:39.171121

Final configuration: 2 counterclockwise turns from full clockwise. Scnitalltors vertically stacked and covered with blanked 


2021-02-07 14:20:53.416676
2021-02-07 14:20:53.996104
2021-02-07 14:20:55.135051
2021-02-07 14:20:58.658662
2021-02-07 14:21:00.292719
2021-02-07 14:21:02.410854
2021-02-07 14:21:02.646239
2021-02-07 14:21:03.016253
2021-02-07 14:21:04.423173
2021-02-07 14:21:05.002635

This is giving me a count rate of about 1.2 Hz. Off of quick searching, i can see that about one cosmic ray per second will reach the detector. (https://home.cern/science/physics/cosmic-rays-particles-outer-space#:~:text=Unlike%20pions%2C%20these%20do%20not,size%20of%20a%20person's%20head.) Thus I am happy with these results and will move on to setting up other aspects. 

### [RN] 7:54 pm 

https://forecast.weather.gov/data/obhistory/metric/KSBA.html
This looks like a good resource to pull weather data. I can bin incident rays into counts per minute for the hour, scrape data off this site, and then update my github. 


I just need to run the script every hour on the hour. after the hour the script will scrape from the NOAA site to update that data (just pull from the table what is unique). It will give data that is number of counts per minute for the whole hour, such that you can make a histogram of the hour. It will run for weeks doing this. Each hour I will also use git to update the repo. 